In [52]:
import sys
sys.path.append('../_amigocloud')

In [53]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [54]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [55]:
# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [56]:
path_xlsx_parte = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [57]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [58]:
consulta = {'query': 'select * from dataset_342996 order by idd'}
seleccion = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', consulta)
datos = seleccion['data']

In [59]:
df_datos = gpd.GeoDataFrame(datos)

In [60]:
list(set(df_datos['idd']))

[568]

In [61]:
524, 533, 535, 536, 539, 540, 541

(524, 533, 535, 536, 539, 540, 541)

In [67]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry) WHERE idd in (568)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [68]:
df_lib = pd.DataFrame(data)

In [69]:
len(df_lib)

1

In [70]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[568]

In [71]:
lista_id = list(set(df_lib['id']))
lista_id

[5050]

In [72]:
nulos = df_lib.isnull().sum()
nulos

id                     0
idd                    0
fecha_registro         0
piloto_1               0
piloto_2               0
hora_inicio            0
hora_fin               0
labor                  0
temperatura            0
viento                 0
humedad                0
estado_de_operacion    0
dron                   0
obs                    1
dtype: int64

In [73]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [74]:
if null_check:
    df_lib = None
else:
    None

In [75]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,5050,568,2025-03-13 13:13:43+00:00,GABRIEL PEÑA,GABRIEL PEÑA,2025-03-13 13:13:43+00:00,2025-03-13 13:16:43+00:00,LIBERACION,24,1,70,CONCLUIDO,DA 02,None


In [76]:
df_parte = pd.read_excel(path_xlsx_parte)

In [77]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE,NUM VISITA
0,641,2024-05-23 00:00:00,21.0,62,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERANZA--SUAREZ DAVID,L1,17.000000,...,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI,1.0
1,558,2024-06-27 00:00:00,26.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C10.3,2.490000,...,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,PROPIO,NaN,NaN,NaN,SI,1.0
2,558,2024-06-27 00:00:00,26.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C12.2,0.830000,...,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,PROPIO,NaN,NaN,NaN,SI,1.0
3,558,2024-06-27 00:00:00,26.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C13.1,0.370000,...,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,PROPIO,NaN,NaN,NaN,SI,1.0
4,558,2024-06-27 00:00:00,26.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C13.2,0.760000,...,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,PROPIO,NaN,NaN,NaN,SI,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,2025-03-14 04:00:00,11.0,81,42239,CORDOVA OLGUIN LEONARDO,636,SAN JUAN--CORDOVA,L3,5.844763,...,EDUARDO REYES,EDUARDO REYES,NaN,NaN,SIEMBRA 2024,5046.0,139.0,553.0,NaN,NaN
4568,NaN,2025-03-14 04:00:00,11.0,81,42239,CORDOVA OLGUIN LEONARDO,367,LA AURORA Y SAN JUAN--CORDOVA,L1,7.929320,...,EDUARDO REYES,EDUARDO REYES,NaN,NaN,SIEMBRA 2024,5047.0,139.0,563.0,NaN,NaN
4569,NaN,2025-03-12 04:00:00,11.0,40,8814,MURIALDO MENDEZ WALTER DANIEL,222,POTRERITO,L52,3.737720,...,GABRIEL PEÑA,GABRIEL PEÑA,NaN,NaN,SIEMBRA 2024,5048.0,139.0,562.0,NaN,NaN
4570,NaN,2025-03-12 04:00:00,11.0,40,8814,MURIALDO MENDEZ WALTER DANIEL,222,POTRERITO,L42,8.633852,...,GABRIEL PEÑA,GABRIEL PEÑA,NaN,NaN,SIEMBRA 2024,5049.0,139.0,562.0,NaN,NaN


In [78]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [79]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

In [80]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [81]:
lista_idd

[568]

In [82]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 568


In [83]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 5050


## Copiar ids de PARTE DIARIO a PLANIFICACION

In [84]:
path_planificacion = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\PLANIFICACION.shp'
path_xlsx = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [85]:
gdf_plan = gpd.read_file(path_planificacion)
df_xlsx = pd.read_excel(path_xlsx)

In [86]:
# Recorrer cada registro de df_xlsx
for index, row in df_xlsx.iterrows():
    id_value = row['ID']
    idd_value = row['IDD']
    
    # Actualizar el campo 'idd' en gdf_plan donde coincida el campo 'id'
    gdf_plan.loc[gdf_plan['id'] == id_value, 'idd'] = idd_value

In [87]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')